In [1]:
# the purpose of this notebook is to try and create a vector DB on the contents of the AI Policy.

In [1]:
# use the beautiful soup loader
from langchain.document_loaders import BSHTMLLoader

In [2]:
# how is the doc being embedded
from langchain.embeddings import OpenAIEmbeddings

In [3]:
# how will text be split?
from langchain.text_splitter import CharacterTextSplitter

In [4]:
# now the database
from langchain.vectorstores import Chroma

In [5]:
from langchain.chat_models import ChatOpenAI

In [82]:
# first load the document
loader = BSHTMLLoader('./policy.html')
documents = loader.load()

In [83]:
# split the docs into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
split_docs = text_splitter.split_documents(documents)

In [6]:
# now connect to embedding function
embedding_function = OpenAIEmbeddings()

In [7]:
# load back the doc
db_new_connection = Chroma(
    persist_directory='.',
    embedding_function=embedding_function
)

In [10]:
# find similar text first
question = input() #'Who and which organizations helped create the policy?'

 At what size will the government like to regulate the AI model?


In [14]:
# we need a MultiQuery
from langchain.retrievers.multi_query import MultiQueryRetriever

In [15]:
# finally, connnect the llm and use it for query
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db_new_connection.as_retriever(),
    llm = llm
)

In [16]:
# get more logging in output
# logging behind scenes
import logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [17]:
# finally execute the chat query and print the result
unique_docs = retriever_from_llm.get_relevant_documents(question)

INFO:langchain.retrievers.multi_query:Generated queries: ["1. What are the government's preferences regarding the regulation of AI models in terms of size?", '2. How does the government determine the size at which they would like to regulate AI models?', "3. What factors influence the government's decision to regulate AI models and at what size do they typically intervene?"]


In [18]:
# finally, summarize the text that was retrieved
matching_docs = ''

for doc in unique_docs:
    matching_docs += doc.page_content

In [19]:
# now execute a new query to LLM
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, ChatPromptTemplate

In [20]:
system_template = 'You are an expert and analyzing the given text input and extracting the relevant information. Answer the user question based only the text provided and no external data'
system_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [21]:
human_message = '''Please answer my {question}. Here is the relevant information below: 
```
{relevant_information}
```
'''
human_prompt = HumanMessagePromptTemplate.from_template(human_message)

In [22]:
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
request = chat_prompt.format_prompt(question=question,relevant_information=matching_docs ).to_messages()
#print(request)

In [23]:
response = llm(request)

In [24]:
print(response.content)

Based on the information provided, the government will like to regulate AI models that pose a serious risk to national security, national economic security, or national public health and safety. Companies developing such AI models will be required to notify the federal government when training the model and share the results of all red-team safety tests. The government will also establish standards, tools, and tests to ensure the safety, security, and trustworthiness of AI systems. Additionally, the government will develop strong new standards for biological synthesis screening to protect against the risks of using AI to engineer dangerous biological materials. The Department of Commerce will establish standards and best practices for detecting AI-generated content and authenticating official content to protect against AI-enabled fraud and deception.
